In [1]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys
import math
import numpy as np
sys.path.append('../pytools')
import d2l

d2l.gpu_mem_init()

2025-09-19 15:53:34.081873: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-19 15:53:34.143625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-19 15:53:35.616036: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): True


In [2]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(name='flatten0'),
    tf.keras.layers.Dense(4, activation=tf.nn.relu, name='Dense0'),
    tf.keras.layers.Dense(4, activation=tf.nn.relu, name='Dense1'),
    tf.keras.layers.Dense(1,name='Dense2'),
], name='Out')

X = tf.random.uniform((2, 4))
net(X)

I0000 00:00:1758268416.595800 3575871 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30495 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:00:08.0, compute capability: 7.0
2025-09-19 15:53:36.841060: W external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:237] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 7.0
2025-09-19 15:53:36.841095: W external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:240] Used ptxas at /usr/local/cuda/bin/ptxas
2025-09-19 15:53:36.841171: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-09-19 15:53:36.844768: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:188] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-09-19 15:53:36.846741: W tensorfl

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>

In [3]:
print(net.layers[0])
print(net.layers[1].weights)
print(net.layers[2].weights)
print(net.layers[3].weights)

<Flatten name=flatten0, built=True>
[<Variable path=Out/Dense0/kernel, shape=(4, 4), dtype=float32, value=[[ 0.23205775  0.31370646  0.55850834  0.15237123]
 [-0.2567699  -0.2192471   0.38721746  0.7242524 ]
 [ 0.14606684  0.6064181   0.3240494   0.19513768]
 [ 0.04005933 -0.59194636 -0.14155227 -0.44525963]]>, <Variable path=Out/Dense0/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]
[<Variable path=Out/Dense1/kernel, shape=(4, 4), dtype=float32, value=[[ 0.37543422  0.13030589 -0.71500844 -0.8276961 ]
 [ 0.06477106 -0.09510988 -0.1312887   0.15442067]
 [-0.21385974  0.18901962  0.15185004 -0.8493795 ]
 [-0.46930444 -0.81070864 -0.40376592 -0.417464  ]]>, <Variable path=Out/Dense1/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]
[<Variable path=Out/Dense2/kernel, shape=(4, 1), dtype=float32, value=[[ 0.5050509 ]
 [-0.54444116]
 [ 0.57134926]
 [ 0.4959364 ]]>, <Variable path=Out/Dense2/bias, shape=(1,), dtype=float32, value=[0.]>]


In [4]:
print(type(net.layers[1].weights[0]))
print(net.layers[1].weights[1])
print(tf.convert_to_tensor(net.layers[1].weights[0]))

<class 'keras.src.backend.Variable'>
<Variable path=Out/Dense0/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>
tf.Tensor(
[[ 0.23205775  0.31370646  0.55850834  0.15237123]
 [-0.2567699  -0.2192471   0.38721746  0.7242524 ]
 [ 0.14606684  0.6064181   0.3240494   0.19513768]
 [ 0.04005933 -0.59194636 -0.14155227 -0.44525963]], shape=(4, 4), dtype=float32)


In [5]:
print(net.get_weights()[2])

[[ 0.37543422  0.13030589 -0.71500844 -0.8276961 ]
 [ 0.06477106 -0.09510988 -0.1312887   0.15442067]
 [-0.21385974  0.18901962  0.15185004 -0.8493795 ]
 [-0.46930444 -0.81070864 -0.40376592 -0.417464  ]]


In [6]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential(name='block2-out')
    for i in range(4):
        # 在这里嵌套
        net.add(block1(name=f'block2-in-{i}'))
    return net

rgnet = tf.keras.Sequential(name='Out')
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.        ],
       [-0.02992861]], dtype=float32)>

In [7]:
#print(rgnet.summary())

In [8]:
print(rgnet.layers[0].layers[1].layers[0].path)

Out/block2-out/block2-in-1/flatten_1


In [9]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data = tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor = (tf.abs(data) > 5)
        factor = tf.cast(factor, dtype=tf.float32)
        return data * factor

In [10]:
net = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation='relu', kernel_initializer=MyInit),
    tf.keras.layers.Dense(1)
])

x = tf.random.uniform((2,20))
net(x)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[31.270807 ],
       [10.2791195]], dtype=float32)>

In [25]:
# tf.keras的表现有点不同。它会自动删除重复层
# 实际测试看tf2.20 & keras 3.11.3并不会删除重复层
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# 检查参数是否不同
#print(len(net.layers))
#print(net.layers[1].weights)
print(net.layers[2].weights)
#print(net.layers[3].weights[0].device)

[<Variable path=sequential_15/dense_35/kernel, shape=(4, 4), dtype=float32, value=[[-0.8111685  -0.69305027 -0.6943098   0.45135564]
 [-0.16501623 -0.13279682  0.5357911   0.11565661]
 [-0.13937807 -0.71508384 -0.12512231  0.5382703 ]
 [ 0.5758949  -0.14786196  0.6496212   0.1637854 ]]>, <Variable path=sequential_15/dense_35/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]


In [ ]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [ ]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [ ]:
x = tf.random.uniform((2,20))
net(x)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]